In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_raw=pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
df=df_raw.copy()
df.head()

In [4]:
df.drop(['center_id','patient_id','image_num'],axis=1,inplace=True)

In [5]:
df.head()

In [6]:
train_folder=[]
for i in df['image_id']:
    train_folder.append(i)

In [7]:
id=[]
for i in train_folder:
    id.append(i+'.tif')
df['id']=id

In [8]:
df.head()

In [9]:
train_dir='/kaggle/input/mayo-clinic-strip-ai/train/'
DataGen=ImageDataGenerator(rescale=1/255,validation_split=0.25)

In [10]:
train_itt=DataGen.flow_from_dataframe(dataframe=df,directory=train_dir,x_col='id',y_col='label',
                                      target_size=(100,100),
                                     subset='training')
Val_itt=DataGen.flow_from_dataframe(dataframe=df,directory=train_dir,x_col='id',y_col='label',
                                     target_size=(100,100),
                                    subset='validation')

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

In [12]:
model= Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(100,100,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))


model.add(Flatten())

model.add(Dense(512,activation='relu'))

model.add(Dense(2,activation='softmax'))

In [13]:
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_itt,validation_data=Val_itt,epochs=2)

In [ ]:
tf

In [ ]:
predictions = model.predict(test_itt)
predictions

In [ ]:
submission = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/sample_submission.csv')

ce = []
laa = []

for pred in predictions:
    ce.append(pred[0])
    laa.append(pred[1])

(submission['CE'],submission['LAA'])  = (ce,laa)

In [ ]:
submission.to_csv('submission.csv',index = False)